In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 125.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers -q

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd

import string
import re
import nltk
from nltk.corpus import stopwords

from google.colab import drive
drive.mount('/drive')

from torch.optim import Adam
import torch
from torch.utils.data import DataLoader

Mounted at /drive


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens(["<response>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 768)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model = model.to(device)

In [ ]:
train = pd.read_csv('/drive/My Drive/Colab Notebooks/Upwork LLM eval/train.csv')
test = pd.read_csv('/drive/My Drive/Colab Notebooks/Upwork LLM eval/test.csv')

In [ ]:
train['input'] = "<startofstring> "+ train['Questions'] +" <response>: "+ train['Answers'] +" <endofstring>"
text_input = list(train['input'])

In [ ]:
from torch.utils.data import Dataset

class ChatData(Dataset):
    def __init__(self, data_in:list, tokenizer):
        self.data = data_in

        # Set pad_token if not set
        if tokenizer.pad_token is None:
          tokenizer.pad_token = tokenizer.eos_token

        self.X_encoded = tokenizer(self.data,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
chatData = ChatData(text_input, tokenizer)
chatData =  DataLoader(chatData, batch_size=8)

In [ ]:
def train_model(chatData,model,optim):
  epochs = 10

  for i in range(epochs):
    for x,a in chatData:
      x = x.to(device)
      a = a.to(device)
      optim.zero_grad()
      loss = model(
          x,
          attention_mask = a,
          labels = x
      ).loss
      loss.backward()
      optim.step()
    torch.save(model.state_dict(),"model_state.pt")
    print(infer("How do I communicate data-driven insights to stakeholders within my startup?"))

In [ ]:
def infer(inp):
  inp = "<startofstring> "+inp+" <response>: "
  inp = tokenizer(inp, return_tensors="pt")
  X = inp["input_ids"].to(device)
  a = inp["attention_mask"].to(device)
  output = model.generate(X, attention_mask=a,max_length=80)
  output = tokenizer.decode(output[0], skip_special_tokens=True)
  return output

In [ ]:
model.train()
optim = Adam(model.parameters())
train_model(chatData,model,optim)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How do I communicate data-driven insights to stakeholders within my startup? <response>: Use data analytics tools to track and analyze data.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How do I communicate data-driven insights to stakeholders within my startup? <response>: Use data analytics tools to track and visualize data.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

How do I communicate data-driven insights to stakeholders within my startup? <response>: -Tailor the communication of insights to the specific needs and interests of each stakeholders.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How do I communicate data-driven insights to stakeholders within my startup? <response>: -Tailor the presentation of the insights to the specific needs and interests of different stakeholders.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How do I communicate data-driven insights to stakeholders within my startup? <response>: Provide valuable content through blogs, videos, or webinars.


In [ ]:
test['gpt2'] = test.apply(lambda x: infer(x['Questions']), axis = 1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
test.to_csv('/drive/My Drive/Colab Notebooks/Upwork LLM eval/test.csv')